# SVN Reports Dashboard
Reporting import/upload pipelines

### Import Libraries
Libraries necessary to parse through each dashboard

In [1]:
import datetime
from IPython.display import display, Markdown, Latex
import json
import numpy as np
import pandas as pd

### Parametrization
Adding parameters for Papermill

In [2]:
site: str

In [3]:
# Parameters
site = "duke"


### Setting Column Names
Use an existing JSON file

In [4]:
def get_special_columns(file_path):
    f = open(file_path, "r")
    data = json.load(f)
    return data[0]

new_column_names = get_special_columns("reference/svn_col_names.json")

### Set up Table Styles
Use `pandas.style` to set up some graphical styles for the main reports table.

In [5]:
table_styles = [{
    'selector': 'td',
    'props': [('background-color', '#FFFFFF'), ('color', '#000000')]
}, 
{
    'selector': 'th',
    'props': [('background-color', '#000000'), ('color', '#FFFFFF')]
}]

def style_row(x):
    # For the sla_percentage: row == 3
    style = [None] * x.size
    sample_series = pd.DataFrame(data=x).iloc[3]
    for index, value in sample_series.items():
        if (value > 1):
            style[3] = "color: red !important;"
            if (value > 10):
                style[3] = style[3] + " font-weight: bold;"
    return style

### Define Functions
Define functions to parse through each of the sites and print them to the screen

In [6]:
# In future iterations, display path is:  '../../../../../log/status_reports/'
def load_dataframe_and_display(file_name=None):
    # Header and load in dataframe
    display(Markdown('## SLA ' + file_name + ' Dashboard'))
    df = pd.read_csv('status_reports/' + file_name.lower() + '.csv', parse_dates=['date_updated'])
    durations = df['sla'].unique().tolist()
    durations = np.sort(durations)
    
    # Categorize each report by SLA (3, 30, 3000 -> Dead)
    for duration in durations:
        # Find specific data subset and 
        duration_df = df.loc[df['sla'] == duration] # Find specific dataframe
        duration_df = duration_df.rename(columns={'laptop':'Laptop'}, errors="raise")
        
        # Create header
        duration_str = str(duration) + '-Day'
        if (duration == 3000):
            duration_str = 'Dead'
        display(Markdown(duration_str + ' SLA Laptop Report'))
        
        # Set index and update date format (latter is hacky -> can fix)
        duration_df = duration_df.set_index(['Laptop'])
        if (duration == 3):
            duration_df['date_updated'] = duration_df['date_updated'].dt.strftime('%Y-%m-%d %H:%M')
            duration_df['time_diff'] = duration_df['time_diff'].apply(lambda x: str(x)[:str(x).find(".") - 3])
        else:
            duration_df['date_updated'] = duration_df['date_updated'].dt.strftime('%Y-%m-%d')
            duration_df['time_diff'] = duration_df['time_diff'].apply(lambda x: str(x)[:str(x).find("days") + len("days")])
            
        # Rename columns, style, and display
        duration_df = duration_df.rename(columns=new_column_names)
        se = duration_df.style.set_table_styles(table_styles).apply(lambda x: style_row(x), axis=1)
        display(se)
        
def main():
    load_dataframe_and_display(site)

### Run all
Run functions and produce dashboards for each site

In [7]:
main()

## SLA duke Dashboard

30-Day SLA Laptop Report

,Date Updated,Time Difference,SLA,SLA %
Laptop,,,,
duke30,2018-04-10,1238 days,30,41.270811
duke19,2018-04-17,1230 days,30,41.033103
duke18,2018-04-28,1219 days,30,40.663691
duke7,2019-03-18,896 days,30,29.868915
duke31,2019-03-18,895 days,30,29.865957
duke36,2020-03-20,528 days,30,17.603372
duke50,2020-05-12,474 days,30,15.830962
duke51,2020-05-13,474 days,30,15.802241


Dead SLA Laptop Report

,Date Updated,Time Difference,SLA,SLA %
Laptop,,,,
duke-merge,2015-03-31,2343 days,3000,0.781261
duke8,2018-04-10,1238 days,3000,0.412691
